In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=05687e49eb26a6bc7753060ab99ec0bc126d08ae31e85558c978e0424bff2b62
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
import numpy as np
import face_recognition
import os
import cv2
from datetime import datetime
import matplotlib.pyplot as plt

path = r'/content/drive/MyDrive/Dataset'
images = []
classNames = []
myList = os.listdir(path)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

def findEncodings(images):
    encodeList = []
    for img in images:
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encode = face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
    return encodeList

def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = [entry.split(',')[0] for entry in myDataList]
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

encodeListKnown = findEncodings(images)
print('Encoding Done')
# Process a single image
img = cv2.imread('/content/drive/MyDrive/Dataset/Dataset/Blessy.jpg')  # Replace with your image path
if img is not None:
    imgS = cv2.resize(img, (int(img.shape[1] * 0.25), int(img.shape[0] * 0.25)), None, fx=0.25, fy=0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    if len(facesCurFrame) > 0:  # Check if faces are detected
        encodeCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace, faceLoc in zip(encodeCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)

            # Check if at least one face is detected
            if facesCurFrame:
                faceDist = face_recognition.face_distance(encodeListKnown, encodeFace)

                if len(faceDist) > 0:
                    accu = str(int(min(faceDist) * 100))
                    matchIndex = np.argmin(faceDist)

                    if matches[matchIndex]:
                        name = classNames[matchIndex].upper()
                        name_accu = name + ',' + accu + '%'

                        y1, x2, y2, x1 = faceLoc
                        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                        cv2.rectangle(img, (x1, y2 - 35), (x2 + 60, y2), (0, 0, 0), cv2.FILLED)
                        cv2.putText(img, name_accu, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                        markAttendance(name)
                else:
                    print("No face distance found.")
            else:
                print("No face detected in the current frame.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dataset'

In [ ]:
!pip install dlib
!pip install matplotlib

import dlib
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Path to the shape predictor model
predictor_model = "/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat"

# Load the image
file_name = '/content/drive/MyDrive/Dataset/Blessy.jpg'  # Replace with your image path
image = io.imread(file_name)

# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()
face_pose_predictor = dlib.shape_predictor(predictor_model)

# Run the HOG face detector on the image data
detected_faces = face_detector(image, 1)

print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

# Loop through each face found in the image
for i, face_rect in enumerate(detected_faces):
    # Draw a rectangle around the face
    rect = patches.Rectangle((face_rect.left(), face_rect.top()), face_rect.width(), face_rect.height(), linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

    # Get the face's pose landmarks
    pose_landmarks = face_pose_predictor(image, face_rect)

    # Plot the landmarks
    for point in range(pose_landmarks.num_parts):
        ax.plot(pose_landmarks.part(point).x, pose_landmarks.part(point).y, 'bo', markersize=2)

# Show the plot
plt.show()

FileNotFoundError: No such file: '/content/drive/MyDrive/Dataset/DEBASISH SAHA.jpg'